In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import json

# local modules 
import sys
sys.path.insert(0, "../scripts")
from helpers import *

In [2]:
a00, a01 = import_desired_data("A", "15T")
b00, b01 = import_desired_data("A", "15T")

In [3]:
r00

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:15:00,23.1806,51.421100,0.0,14.333333,30.555556,1.0
1,2022-07-20 07:30:00,23.3258,51.759333,0.0,14.333333,30.555556,1.0
2,2022-07-20 07:45:00,23.4242,51.619233,0.0,14.333333,30.555556,1.0
3,2022-07-20 08:00:00,23.5314,51.218100,0.0,14.666667,30.555556,1.0
4,2022-07-20 08:15:00,23.6314,50.950367,0.0,14.666667,30.555556,1.0
...,...,...,...,...,...,...,...
380,2022-07-24 06:15:00,23.2674,51.580067,0.0,14.611111,30.555556,1.0
381,2022-07-24 06:30:00,23.2762,51.851233,0.0,14.611111,30.555556,1.0
382,2022-07-24 06:45:00,23.2850,51.867167,0.0,14.611111,30.555556,1.0
383,2022-07-24 07:00:00,23.2850,51.872333,0.0,14.722222,30.555556,1.0


In [5]:
r01["Temp C"].max()

25.5085

In [13]:
win_norm0 = normalize_scale(r01["Window Open"], r01["Temp C"].min(), r01["Temp C"].max() )

In [38]:
def normalize_window_data(df):
    if len(df["Window Open"].unique()) > 1:
        return normalize_scale(df["Window Open"], df["Temp C"].min(), df["Temp C"].max())  
    else:
        return df["Window Open"]*df["Temp C"].max()

In [39]:
normalize_window_data(r00)

1


0      25.6325
1      25.6325
2      25.6325
3      25.6325
4      25.6325
        ...   
380    25.6325
381    25.6325
382    25.6325
383    25.6325
384    25.6325
Name: Window Open, Length: 385, dtype: float64

In [16]:
len(r00["Window Open"].unique())

1

In [102]:
def add_var(df):
    if len(df["Window Open"].unique()) == 1:
        df["Window Open"][0] = 1 if df["Window Open"][0] == 0 else 0
    return df


In [103]:
add_var(r00)

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:15:00,23.1806,51.421100,0.0,14.333333,30.555556,0.0
1,2022-07-20 07:30:00,23.3258,51.759333,0.0,14.333333,30.555556,1.0
2,2022-07-20 07:45:00,23.4242,51.619233,0.0,14.333333,30.555556,1.0
3,2022-07-20 08:00:00,23.5314,51.218100,0.0,14.666667,30.555556,1.0
4,2022-07-20 08:15:00,23.6314,50.950367,0.0,14.666667,30.555556,1.0
...,...,...,...,...,...,...,...
380,2022-07-24 06:15:00,23.2674,51.580067,0.0,14.611111,30.555556,1.0
381,2022-07-24 06:30:00,23.2762,51.851233,0.0,14.611111,30.555556,1.0
382,2022-07-24 06:45:00,23.2850,51.867167,0.0,14.611111,30.555556,1.0
383,2022-07-24 07:00:00,23.2850,51.872333,0.0,14.722222,30.555556,1.0


In [98]:
r00["Window Open"].iloc[0] = 0

/var/folders/7r/39t1cgb13d57zk26c__2z50m0000gn/T/ipykernel_16357/832304184.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [104]:
r00["DateTime"].iloc[-1]

Timestamp('2022-07-24 07:15:00')

In [119]:
def make_traces(fig, df, row, col):
    df = add_var(df)
    fig.add_trace(
        go.Scatter(
            x=df["DateTime"],
            y=df["Temp C"], 
            mode='lines',
            line=dict(width=4),
            showlegend=False
        ),
        row=row, col=col,
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(
            x=r01["DateTime"],
            y=df["Window Open"], 
            # y=normalize_window_data(df),
            mode='lines',
            marker_color="gray",
            showlegend=False
        ),
        row=row, col=col,
        secondary_y=True,
    )

    fig.update_layout(
        yaxis2=dict(
            tickmode = 'array',
            tickvals = [0,1],
            ticktext = ['Closed', 'Open']
        ),
        xaxis_range=[df["DateTime"][2], df["DateTime"].iloc[-1]],
    )



    return fig


In [121]:
fig = make_subplots(rows=3, cols=2, 
            specs=[[{"secondary_y": True}, {"secondary_y": True}],
                   [{"secondary_y": True}, {"secondary_y": True}],
                    [{"secondary_y": True}, {"secondary_y": True}]])
fig = make_traces(fig, r00, row=1, col=1)
fig = make_traces(fig, r01, row=1, col=2)
fig = make_traces(fig, r01, row=2, col=1)
fig

In [11]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, subplot_titles=("Room A", "Room B"))

fig.add_trace(go.Scatter(
    x=r01["DateTime"],
    y=r01["Temp C"], 
    mode='lines',
    line=dict(width=4)
))
fig.add_trace(go.Scatter(
    x=r01["DateTime"],
    y=win_norm, 
    mode='lines',
    marker_color="gray"
))